In [168]:
import pandas as pd
import numpy as np

# Load the data generated from the R code

In [222]:
go_bp = pd.read_csv("./GO_BP.csv")
go_bp = go_bp.drop(labels=['EVIDENCE', 'ENTREZID', 'ONTOLOGY'], axis=1)
go_bp = go_bp.drop_duplicates()
go_bp.head()

,ENSEMBL,GO
0,ENSG00000000003,GO:0039532
1,ENSG00000000003,GO:0043123
2,ENSG00000000003,GO:1901223
3,ENSG00000000005,GO:0001886
4,ENSG00000000005,GO:0001937


In [170]:
counts1 = pd.read_csv("./counts1.csv", index_col=0)
counts1.head()

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSGR0000167393,ENSGR0000169084,ENSGR0000169093,ENSGR0000178605,ENSGR0000182378,ENSGR0000185291,ENSGR0000198223,ENSGR0000214717,ENSGR0000223511,ENSGR0000223773
089357B,14,7,103,241,72,2057,30,60,207,367,...,1,0,0,0,0,0,0,0,0,0
089366A,11,2,194,511,110,3325,36,111,186,530,...,0,0,0,0,0,0,1,0,0,1
089412B,8,0,312,450,106,3751,45,160,325,653,...,0,0,0,0,0,0,1,0,0,0
089425B,9,0,135,496,133,2758,26,93,182,620,...,0,0,0,0,0,0,0,0,0,0
089687A,4,0,89,267,49,2181,24,75,122,263,...,0,0,0,0,0,0,1,0,0,0


In [171]:
pheno1 = pd.read_csv("./pheno1.csv", index_col=0)
pheno1.head()

,age,diagnosis,sex,lithium,condition
089357B,18,Control,F,0,Control
089366A,19,Control,F,0,Control
089412B,23,Control,F,0,Control
089425B,47,Control,F,0,Control
089687A,52,Control,F,0,Control


# Data Exploration Analysis

In [172]:
available_genes = set(go_bp.ENSEMBL)
("length of all_genes :", len(available_genes))

('length of all_genes :', 17599)

In [173]:
data_genes = set(counts1.columns)
("length of data_genes :", len(data_genes))

('length of data_genes :', 52645)

In [174]:
("Number of genes missing :", len(data_genes - available_genes))

('Number of genes missing :', 35046)

In [175]:
("Ratio of genes missing : {:.2f}%".format(len(data_genes - available_genes) / len(data_genes) * 100))

'Ratio of genes missing : 66.57%'

# GO layers architecture

## load GO graph

In [176]:
import obonet
import networkx as nx

In [177]:
# Read the gene ontology
!wget http://purl.obolibrary.org/obo/go/go-basic.obo 
original_graph = obonet.read_obo('go-basic.obo')
!rm go-basic.obo

--2024-04-18 23:08:22--  http://purl.obolibrary.org/obo/go/go-basic.obo
Resolving purl.obolibrary.org (purl.obolibrary.org)... 2606:4700:4400::ac40:96c5, 2606:4700:4400::6812:253b, 104.18.37.59, ...
Connecting to purl.obolibrary.org (purl.obolibrary.org)|2606:4700:4400::ac40:96c5|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://current.geneontology.org/ontology/go-basic.obo [following]
--2024-04-18 23:08:23--  http://current.geneontology.org/ontology/go-basic.obo
Resolving current.geneontology.org (current.geneontology.org)... 2600:9000:24db:e800:d:ff6c:c780:93a1, 2600:9000:24db:0:d:ff6c:c780:93a1, 2600:9000:24db:a200:d:ff6c:c780:93a1, ...
Connecting to current.geneontology.org (current.geneontology.org)|2600:9000:24db:e800:d:ff6c:c780:93a1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31267219 (30M) [text/obo]
Saving to: ‘go-basic.obo’

go-basic.obo        100%[===================>]  29.82M  6.77MB/s    in 4.4s    

2024-0

In [178]:
print(original_graph)

MultiDiGraph named 'go' with 42312 nodes and 82461 edges


## only keep the Biological Process (BP) subgraph

In [179]:
node_BP = "GO:0008150"
nodes_BP = nx.ancestors(original_graph, node_BP)
nodes_BP.add(node_BP)
print("Number of nodes in the graph : ", len(nodes_BP))

Number of nodes in the graph :  27074


In [180]:
graph_BP = original_graph.subgraph(nodes_BP)
nx.is_directed_acyclic_graph(graph_BP)

True

## remove the subgraph which are not in the list of GO terms

,ENSEMBL,GO
0,ENSG00000000003,GO:0039532
1,ENSG00000000003,GO:0043123
2,ENSG00000000003,GO:1901223
3,ENSG00000000005,GO:0001886
4,ENSG00000000005,GO:0001937


In [182]:
terms_go = gene_go_map.GO.unique()
terms_go = list(terms_go)
len(terms_go)

12305

iterate from the leaves to the root, trim all the branches that do not contain any GO term

In [183]:
from tqdm import tqdm

list_nodes_useless_cropped = [x for x in nodes_BP if graph_BP.in_degree(x) == 0 and x not in terms_go]
list_ancestors_to_remove, list_ancestors_to_keep = [], []

for child in tqdm(list_nodes_useless_cropped):
    ancestors = nx.descendants(graph_BP, child)
    for ancestor in ancestors:
        if (list_ancestors_to_remove.count(ancestor) == 0) and (list_ancestors_to_keep.count(ancestor) == 0):
            if terms_go.count(ancestor) == 0:
                descendants = list(nx.ancestors(graph_BP, ancestor))
                if gene_go_map.GO.isin(descendants).sum() == 0:
                    list_ancestors_to_remove.append(ancestor)
                else:
                    list_ancestors_to_keep.append(ancestor)
            else:
                list_ancestors_to_keep.append(ancestor)

100%|██████████| 7255/7255 [02:12<00:00, 54.59it/s]


In [184]:
list_nodes_to_remove = set(list_ancestors_to_remove + list_nodes_useless_cropped)
len(list_nodes_to_remove)

11707

In [185]:
nodes_BP = nodes_BP - list_nodes_to_remove
graph_BP = graph_BP.subgraph(nodes_BP)
nx.is_directed_acyclic_graph(graph_BP)

True

In [186]:
print("Number of nodes in the graph : ", len(nodes_BP))

Number of nodes in the graph :  15367


In [193]:
# import pickle
# 
# with open('gobp-entire.gpickle', 'wb') as f:
#     pickle.dump(graph_BP, f, pickle.HIGHEST_PROTOCOL)

AttributeError: Can't pickle local object 'subgraph_view.<locals>.reverse_edge'

In [188]:
## 

In [203]:
gene_go_map = gene_go_map.drop_duplicates()
gene_go_map = gene_go_map[gene_go_map.GO.isin(graph_BP.nodes)]

,ENSEMBL,GO
0,ENSG00000000003,GO:0039532
1,ENSG00000000003,GO:0043123
2,ENSG00000000003,GO:1901223
3,ENSG00000000005,GO:0001886
4,ENSG00000000005,GO:0001937
...,...,...
182195,ENSG00000273079,GO:1904062
182196,ENSG00000273079,GO:2000463
182198,ENSG00000273079,GO:2001056
182250,ENSG00000273173,GO:0008150


In [205]:
gp = gene_go_map.groupby("ENSEMBL")
for ensembl in tqdm(list(gp.groups.keys())):
    go_associated = gp.get_group(ensembl)["GO"].tolist()
    indexes = list(gp.groups[ensembl])
    current_idx = 0
    while current_idx < len(go_associated):
        go_visited = go_associated[current_idx]
        idx = indexes.pop(0)
        successors = set(nx.ancestors(graph_BP, go_visited))
        if len(successors.intersection(set(go_associated))) != 0:
            file_go = gene_go_map.drop(idx)
            keep1 = go_visited
            keep2 = idx
        current_idx += 1

100%|██████████| 17589/17589 [06:55<00:00, 42.29it/s] 


In [206]:
gene_go_map.to_csv("GOannotations_kept.csv")

In [ ]:
## 

In [207]:
gp_go = gene_go_map.groupby("GO")

In [212]:
final_genes = list(data_genes - available_genes)

In [215]:
len(final_genes), len(gp_go.groups.keys())

(35046, 12114)

In [213]:
matrix_connection = pd.DataFrame(index=final_genes, columns=list(gp_go.groups.keys()))
matrix_connection = matrix_connection.fillna(0)
matrix_connection.head()

KeyboardInterrupt: 

In [ ]:
matrix_connection.shape

In [ ]:
matrix_connection.to_csv(os.path.join(dir_files,"matrix_connection_entire.csv"))

## GO graph statistics 

In [ ]:
in_d = [x[1] for x in graph_BP.in_degree()]
out_d = [x[1] for x in graph_BP.out_degree()]
all_d = [x[1] for x in graph_BP.degree()]
levels = [nx.shortest_path_length(graph_BP, node, node_BP) for node in graph_BP.nodes]
go_nodes = list(graph_BP.nodes)
dfInfoGraph = pd.DataFrame({"d-": in_d, "d+": out_d, "d": all_d, "level": levels}, index=go_nodes)
dfInfoGraph.head()

In [ ]:
dfInfoGraph.describe()

In [ ]:
import matplotlib.pyplot as plt

dfInfoGraph.groupby("level").size().plot(kind='bar')
plt.show()